In [2]:
import pandas as pd
import numpy as np
import pickle
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [3]:
in_file = open(r'total_day_supply.pkl')
total_day_supply = pickle.load(in_file)
in_file.close()

In [4]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((loader['data'], 
                       loader['indices'], 
                       loader['indptr']), shape=loader['shape'])

In [5]:
X = load_sparse_csr(r'data_matrix_one-hot.npz')

In [6]:
X_test, X_train, supply_test, supply_train = train_test_split(X,
                                                              total_day_supply,
                                                              test_size=0.15)

In [7]:
clf = RandomForestRegressor(n_jobs=63, n_estimators=140, verbose=2)

In [8]:
clf.fit(X_train, supply_train)

building tree 1 of 140building tree 2 of 140building tree 3 of 140building tree 4 of 140building tree 5 of 140building tree 6 of 140building tree 8 of 140building tree 9 of 140building tree 10 of 140 building tree 11 of 140building tree 12 of 140








building tree 7 of 140

building tree 14 of 140building tree 13 of 140building tree 15 of 140building tree 16 of 140building tree 17 of 140building tree 18 of 140 building tree 20 of 140building tree 21 of 140building tree 23 of 140 building tree 24 of 140building tree 25 of 140building tree 26 of 140building tree 27 of 140building tree 28 of 140building tree 29 of 140 building tree 31 of 140building tree 32 of 140building tree 33 of 140 building tree 35 of 140building tree 36 of 140building tree 37 of 140 building tree 40 of 140   building tree 43 of 140 building tree 45 of 140building tree 46 of 140 building tree 48 of 140building tree 49 of 140 building tree 52 of 140building tree 51 of 140

building tree 50 of 140

building tree 56

[Parallel(n_jobs=63)]: Done  86 out of 140 | elapsed: 330.8min remaining: 207.7min
[Parallel(n_jobs=63)]: Done 140 out of 140 | elapsed: 432.4min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=140, n_jobs=63, oob_score=False, random_state=None,
           verbose=2, warm_start=False)

In [9]:
clf.score(X_train, supply_train)

[Parallel(n_jobs=63)]: Done  86 out of 140 | elapsed:    6.7s remaining:    4.2s
[Parallel(n_jobs=63)]: Done 140 out of 140 | elapsed:    8.7s finished


0.90757789577833525

In [10]:
clf.score(X_test, supply_test)

[Parallel(n_jobs=63)]: Done  86 out of 140 | elapsed:   39.4s remaining:   24.8s
[Parallel(n_jobs=63)]: Done 140 out of 140 | elapsed:   50.5s finished


0.35726939398884572

In [11]:
from sklearn.metrics import mean_squared_error

In [12]:
mean_squared_error(supply_train, clf.predict(X_train))

[Parallel(n_jobs=63)]: Done  86 out of 140 | elapsed:    7.0s remaining:    4.4s
[Parallel(n_jobs=63)]: Done 140 out of 140 | elapsed:    9.0s finished


1567647.3113527221

In [13]:
mean_squared_error(supply_test, clf.predict(X_test))

[Parallel(n_jobs=63)]: Done  86 out of 140 | elapsed:   39.8s remaining:   25.0s
[Parallel(n_jobs=63)]: Done 140 out of 140 | elapsed:   49.9s finished


10900560.620678179

In [14]:
from sklearn.metrics import median_absolute_error

In [15]:
median_absolute_error(supply_train, clf.predict(X_train))

[Parallel(n_jobs=63)]: Done  86 out of 140 | elapsed:    6.9s remaining:    4.3s
[Parallel(n_jobs=63)]: Done 140 out of 140 | elapsed:    9.0s finished


222.12142857142862

In [16]:
median_absolute_error(supply_test, clf.predict(X_test))

[Parallel(n_jobs=63)]: Done  86 out of 140 | elapsed:   39.9s remaining:   25.1s
[Parallel(n_jobs=63)]: Done 140 out of 140 | elapsed:   50.8s finished


604.06607993197281

In [17]:
from sklearn.metrics import mean_absolute_error

In [18]:
mean_absolute_error(supply_train, clf.predict(X_train))

[Parallel(n_jobs=63)]: Done  86 out of 140 | elapsed:    7.1s remaining:    4.4s
[Parallel(n_jobs=63)]: Done 140 out of 140 | elapsed:    9.0s finished


566.7816725134677

In [19]:
mean_absolute_error(supply_test, clf.predict(X_test))

[Parallel(n_jobs=63)]: Done  86 out of 140 | elapsed:   39.2s remaining:   24.6s
[Parallel(n_jobs=63)]: Done 140 out of 140 | elapsed:   50.2s finished


1516.5581085723088

# Create a frame for the predicted values and their differences

In [89]:
supply_train = supply_train.to_frame()

In [90]:
supply_train['predicted'] = clf.predict(X_train)

[Parallel(n_jobs=63)]: Done  86 out of 140 | elapsed:    7.0s remaining:    4.4s
[Parallel(n_jobs=63)]: Done 140 out of 140 | elapsed:    9.0s finished


In [91]:
supply_train.to_pickle('supply_train_frame.pkl')

In [37]:
supply_test = supply_test.to_frame()

In [39]:
supply_test['predicted'] = clf.predict(X_test)

[Parallel(n_jobs=63)]: Done  86 out of 140 | elapsed:   38.6s remaining:   24.2s
[Parallel(n_jobs=63)]: Done 140 out of 140 | elapsed:   49.6s finished


In [40]:
supply_test.head()

,TOTAL_DAY_SUPPLY,predicted
1152150,420,417.957143
7781877,1620,2298.792857
6398326,1020,672.750000
1622989,600,2084.850000
4389735,211,2379.685714


In [43]:
supply_test['difference'] = supply_test.TOTAL_DAY_SUPPLY - supply_test.predicted

In [46]:
supply_test['difference'] = supply_test.difference.apply(lambda x: np.abs(x))

# Recreate source dataframe

In [53]:
payments_types = {'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID': 'str',
 'Charity_Indicator': 'str',
 'City_of_Travel': 'str',
 'Contextual_Information': 'str',
 'Country_of_Travel': 'str',
 'NDC_of_Associated_Covered_Drug_or_Biological1': 'str',
 'NDC_of_Associated_Covered_Drug_or_Biological2': 'str',
 'NDC_of_Associated_Covered_Drug_or_Biological3': 'str',
 'NDC_of_Associated_Covered_Drug_or_Biological4': 'str',
 'NDC_of_Associated_Covered_Drug_or_Biological5': 'str',
 'Name_of_Associated_Covered_Device_or_Medical_Supply1': 'str',
 'Name_of_Associated_Covered_Device_or_Medical_Supply2': 'str',
 'Name_of_Associated_Covered_Device_or_Medical_Supply3': 'str',
 'Name_of_Associated_Covered_Device_or_Medical_Supply4': 'str',
 'Name_of_Associated_Covered_Device_or_Medical_Supply5': 'str',
 'Name_of_Associated_Covered_Drug_or_Biological1': 'str',
 'Name_of_Associated_Covered_Drug_or_Biological2': 'str',
 'Name_of_Associated_Covered_Drug_or_Biological3': 'str',
 'Name_of_Associated_Covered_Drug_or_Biological4': 'str',
 'Name_of_Associated_Covered_Drug_or_Biological5': 'str',
 'Name_of_Third_Party_Entity_Receiving_Payment_or_Transfer_of_Value': 'str',
 'Physician_License_State_code2': 'str',
 'Physician_License_State_code3': 'str',
 'Physician_License_State_code4': 'str',
 'Physician_License_State_code5': 'str',
 'Physician_First_Name': 'str',
 'Physician_Last_Name': 'str',
 'Physician_Middle_Name': 'str',
 'Physician_Name_Suffix': 'str',
 'Physician_Profile_ID': 'str',
 'Recipient_Postal_Code': 'str',
 'Recipient_Primary_Business_Street_Address_Line2': 'str',
 'Recipient_Province': 'str',
 'Recipient_Zip_Code': 'str',
 'Record_ID': 'str',
 'State_of_Travel': 'str',
 'Teaching_Hospital_ID': 'str',
 'Teaching_Hospital_Name': 'str',
 'Third_Party_Equals_Covered_Recipient_Indicator': 'str'}

In [55]:
# load in datasets
payments = pd.read_csv(r'data/OP_DTL_GNRL_PGYR2014_P06302016.csv', dtype=payments_types)

in_file = open(r'data/prescriptions.pkl', 'rb')
prescriptions = pickle.load(in_file)
in_file.close()

file_in = open(r'data/physician_id_to_npi.pkl', 'rb')
physician_id_to_npi = pickle.load(file_in)
file_in.close()

In [56]:
payments_grouped = payments.groupby(['Physician_Profile_ID', 
                                 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name']).agg({'Total_Amount_of_Payment_USDollars': 'sum'})
payments_grouped = payments_grouped.unstack(
    'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name').reset_index().fillna(0)

payments_grouped.columns = payments_grouped.columns.droplevel()
# create a named column for the Physician_Profile_ID
payments_grouped['Physician_Profile_ID'] = payments_grouped[[0]].values
# drop the unnamed column
#payments_grouped.drop(payments_grouped.columns[0], axis=1, inplace=True)

# Insert the NPI into payments
payments_grouped = payments_grouped.merge(physician_id_to_npi, on='Physician_Profile_ID', copy=False)

prescriptions = prescriptions[['NPI', 'SPECIALTY_DESCRIPTION', 'DRUG_NAME', 'TOTAL_DAY_SUPPLY']]
# merge the payments records with prescription data
prescriptions = prescriptions.merge(payments_grouped, on='NPI', copy=False)


In [58]:
prescriptions.head()

,NPI,SPECIALTY_DESCRIPTION,DRUG_NAME,TOTAL_DAY_SUPPLY,,"180 Medical, Inc.",3D Systems,3M Company,"4WEB, Inc.","A-dec, Inc.",...,"eCardio Diagnostics, LLC","iCAD, Inc","iRhythm Technologies, Inc.",iScreen Vision Inc.,integrated dental systems,"nContact Surgical, Inc",optos plc,"rEVO Biologics, Inc.",sanofi-aventis U.S. LLC,Physician_Profile_ID
0,1588763981,Urology,ANDROGEL,720,1063317,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1063317
1,1588763981,Urology,BICALUTAMIDE,6000,1063317,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1063317
2,1588763981,Urology,CEPHALEXIN,160,1063317,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1063317
3,1588763981,Urology,CIPROFLOXACIN HCL,415,1063317,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1063317
4,1588763981,Urology,FINASTERIDE,28140,1063317,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1063317


In [94]:
prescriptions.memory_usage().sum() / (1024**3)

97

# Good predictions

In [64]:
top_20 = supply_test.sort_values('difference').head(20)

In [66]:
top_20.head()

,TOTAL_DAY_SUPPLY,predicted,difference
107189,1200,1200.0,0.0
4536637,367,367.0,0.0
6910698,1676,1676.0,0.0
3020304,2107,2107.0,0.0
1952266,1074,1074.0,0.0


# Bad predictions

In [65]:
bot_20 = supply_test.sort_values('difference', ascending=False).head(20)

In [67]:
bot_20.head()

,TOTAL_DAY_SUPPLY,predicted,difference
6344543,212622,12999.871429,199622.128571
75266,209235,18513.467857,190721.532143
7138920,193753,10632.678571,183120.321429
5390448,188354,8225.464286,180128.535714
7691294,179537,981.835714,178555.164286


So some of these prescription rates it predicted spot on! And some of them were way off. Looks like the most accurate were generally lower values and those that were less accurate were generally higher values, this makes sense because the training data had many more low values than high values.

In [70]:
top_20_indices = list(top_20.index)

In [71]:
bot_20_indices = list(bot_20.index)

In [77]:
payment_columns = prescriptions.columns[5:-1].tolist()
prescriptions['payment_total'] = prescriptions[payment_columns].sum(axis=1)

In [81]:
prescriptions.loc[top_20_indices].to_csv(r'top_20_predicted_values.csv')

In [82]:
prescriptions.loc[bot_20_indices].to_csv(r'bot_20_predicted_values.csv')

In [83]:
top_20.to_csv(r'top_20_values.csv')

In [84]:
bot_20.to_csv(r'bot_20_values.csv')

In [85]:
supply_test.to_pickle(r'predicted_test_values.pkl')

In [20]:
total_day_supply.describe()

count    8.743650e+06
mean     2.306533e+03
std      4.118261e+03
min      1.100000e+01
25%      4.850000e+02
50%      9.900000e+02
75%      2.262000e+03
max      2.436100e+05
Name: TOTAL_DAY_SUPPLY, dtype: float64

In [21]:
4.11e3

4110.0

In [31]:
np.save('feature_importances', clf.feature_importances_)

In [86]:
clf.feature_importances_

array([  2.28776170e-06,   7.37039930e-05,   2.60899731e-04, ...,
         8.44594304e-07,   6.83287332e-09,   2.51751362e-05])